In [ ]:
# These may be needed on new environments
#!pip install cntk --upgrade 
#!pip install keras --upgrade
#!pip install onnxruntime 
#!pip install numpy

In [2]:
# Import libraries
import cntk
import onnx
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import onnxruntime as rt

In [9]:
# Training Set
# Vectors - [height, weight, shoe_size]
X = np.array([
     [181, 80, 44], [177, 70, 43], [160, 60, 38],
     [154, 54, 37], [166, 65, 40], [190, 90, 47],
     [175, 64, 39], [177, 70, 40], [159, 55, 37],
     [171, 75, 42], [181, 85, 43]])

# Labels - 1 per vector
Y = np.array([0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])

# create model
model = Sequential()
model.add(Dense(60, input_dim=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(X, Y, epochs=100, verbose=0)

In [10]:
# save the model in ONNX format
cntk.combine(model.outputs).save("/mount/gender_classifier.onnx", format=cntk.ModelFormat.ONNX)

In [11]:
# Test the ONNX model using the ONNX runtime
sess = rt.InferenceSession("/mount/gender_classifier.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
X_test = np.array([160,60,43])
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]
pred_onx

array([0.49685702], dtype=float32)

In [12]:
# Prepare and check the saved model for serialization
onx = onnx.load('/mount/gender_classifier.onnx')
print(onx)

ir_version: 4
producer_name: "CNTK"
producer_version: "CNTK_VERSION"
domain: "ai.cntk"
model_version: 1
graph {
  node {
    input: "dense_6/bias"
    input: "Reshape986_Output_0_shape"
    output: "Reshape986_Output_0"
    name: "Reshape986"
    op_type: "Reshape"
    doc_string: ""
    domain: ""
  }
  node {
    input: "dense_5/bias"
    input: "Reshape964_Output_0_shape"
    output: "Reshape964_Output_0"
    name: "Reshape964"
    op_type: "Reshape"
    doc_string: ""
    domain: ""
  }
  node {
    input: "dense_5_input"
    input: "dense_5/kernel"
    output: "Times961_Output_0"
    name: "Times961"
    op_type: "MatMul"
    doc_string: ""
    domain: ""
  }
  node {
    input: "Times961_Output_0"
    input: "Reshape964_Output_0"
    output: "Plus967_Output_0"
    name: "Plus967"
    op_type: "Add"
    doc_string: ""
    domain: ""
  }
  node {
    input: "Plus967_Output_0"
    output: "ReLU970_Output_0"
    name: "ReLU970"
    op_type: "Relu"
    doc_string: ""
    domain: ""
  

In [13]:
# Store the model in MarkLogic
import requests
from requests.auth import HTTPDigestAuth

try:
    url = 'http://aidata:8011/v1/documents'
    header = {"Accept": "application/json"}
    params = 'uri=/mount/gender_classifier.onnx'
    
    response = (requests.put(url+"?"+params, auth=HTTPDigestAuth('admin', 'admin'), headers=header, data=onx.SerializeToString()))
    print(response)
except Exception as e:
    raise e

<Response [204]>
